In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

In [20]:
data = pd.read_csv('c172_file_1.csv')
data

,AltAGL,AltB,AltGPS,AltMSL,BaroA,E1_CHT1,E1_CHT2,E1_CHT3,E1_CHT4,E1_EGT1,...,LatAc,NormAc,OAT,Pitch,Roll,TAS,VSpd,VSpdG,WndDr,WndSpd
0,0.0,855.8,745.7,833.6,30.05,231.88,224.27,243.57,245.74,1047.12,...,0.01,0.01,27.2,1.25,-0.21,0.0,-14.35,-3.9,0.0,0.0
1,0.0,856.8,746.2,834.0,30.05,232.23,224.58,243.87,246.04,1046.06,...,0.01,-0.00,27.2,1.23,-0.15,0.0,0.74,-3.9,0.0,0.0
2,0.0,855.8,746.4,834.2,30.05,232.59,224.84,244.12,246.34,1046.17,...,0.00,-0.01,27.2,1.21,-0.14,0.0,5.96,-3.9,0.0,0.0
3,0.0,856.8,745.2,833.1,30.05,232.96,225.10,244.46,246.59,1046.32,...,-0.00,-0.01,27.2,1.22,-0.14,0.0,9.95,-3.9,0.0,0.0
4,0.0,856.8,745.2,833.0,30.05,233.30,225.36,244.75,246.86,1043.91,...,0.01,-0.01,27.2,1.22,-0.10,0.0,13.36,-3.9,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5055,0.0,844.2,751.5,839.3,30.02,282.56,270.85,296.29,288.37,1041.83,...,0.00,0.04,26.0,1.45,0.00,0.0,15.17,-3.9,0.0,0.0
5056,0.0,844.2,751.6,839.4,30.02,282.49,270.90,296.36,288.45,1036.33,...,0.00,-0.02,26.0,1.29,0.03,0.0,15.98,0.0,0.0,0.0
5057,0.0,844.2,751.7,839.5,30.02,282.43,270.93,296.42,288.52,1030.14,...,0.00,0.01,26.0,1.76,0.02,0.0,17.44,0.0,0.0,0.0
5058,1.0,845.2,752.4,840.2,30.02,282.55,271.14,296.58,288.66,1024.47,...,-0.00,-0.01,26.0,1.80,0.01,0.0,25.02,0.0,0.0,0.0


In [21]:
#Split and reshape the data set by step_size , use min-max or stanrdardlize method to rescale the data
def Splitting_dataset(data, step_size, scale=True, scaler_type=MinMaxScaler):
        l = len(data) 
        data = scaler_type().fit_transform(data)
        Xs = []
        Ys = []
        for i in range(0, (len(data) - step_size)):
            Xs.append(data[i:i+step_size])
            Ys.append(data[i:i+step_size])
        train_x, test_x, train_y, test_y = [np.array(x) for x in train_test_split(Xs, Ys)]
        assert train_x.shape[2] == test_x.shape[2] == (data.shape[1] if (type(data) == np.ndarray) else len(data))
        return  (train_x.shape[2], train_x, train_y, test_x, test_y)

def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def loss_fn(recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    
        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu**2 -  logvar.exp())
        return BCE + KLD

In [87]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=27, z_dim=31, n_flow_steps=1):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, h_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(h_dim, z_dim*2) #is it saying its getting a mu and a var for each z dim out?
            
            #how can I represent the encoder as a distribution acting as the prior?
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, image_size),
            nn.Sigmoid()
        )
        
        #self.flow = stacked_NVP(z_dim,)
        
        #should i add a flow model as a paremeter of the vae for easy access in parameterize?
    
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_() 
        esp = to_var(torch.randn(*mu.size()))
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        #print("esp: " + str(esp))
        #randn - Returns a tensor filled with random numbers from a normal distribution with mean 0 and variance 1 (also called the standard normal distribution).
        z = mu + std * esp
        #print(z)
        return z
    
    def forward(self, x):
        h = self.encoder(x)
        #print("h: " + str(h))
        mu, logvar = torch.chunk(h, 2, dim=1)
        #reparameterize is doing the sampling with esp
        z = self.reparameterize(mu, logvar)
        #print("z: " + str(z))

        prior = torch.distributions.Normal(mu,(logvar.mul(0.5).exp_()))
        ml = nn.ModuleList()
        ml.append(AffineConstantFlow(31))
        ml.append(AffineConstantFlow(31))
        ml.append(AffineConstantFlow(31))
        ml.append(AffineConstantFlow(31))
        #sacf = 

        nf = NormalizingFlowModel(prior,ml)
        zTransformed = nf(z) #this is where I apply the normalizing flow once its implemented, the points are transformed as theyre generated
    
        #print("Z: " + str(z))
        #print("ZT: " + str(zTransformed))
        print(z)
        z = zTransformed[0]
        print(z)
        return self.decoder(z), mu, logvar
    
class NormalizingFlow(nn.Module):
    """ A sequence of Normalizing Flows is a Normalizing Flow """

    def __init__(self, flows):
        super().__init__()
        self.flows = nn.ModuleList(flows)

    def forward(self, x):
        m, _ = x.shape
        log_det = torch.zeros(m)
        zs = [x]
        for flow in self.flows:
            #h = demo.encoder(x)
            #mu, logvar = torch.chunk(h, 2, dim=1)
            x, ld = flow.forward(x)#,demo.reparameterize(mu,logvar))
            log_det += ld
            zs.append(x)
            #here we're just summing the log deterimant of each of the flows
        return zs, log_det

    def backward(self, z):
        m, _ = z.shape
        log_det = torch.zeros(m)
        xs = [z]
        for flow in self.flows[::-1]:
            z, ld = flow.backward(z)
            log_det += ld
            xs.append(z)
        return xs, log_det

class NormalizingFlowModel(nn.Module):
    """ A Normalizing Flow Model is a (prior, flow) pair """
    
    def __init__(self, prior, flows):
        super().__init__()
        self.prior = prior
        self.flow = NormalizingFlow(flows)
    
    def forward(self, x):
        zs, log_det = self.flow.forward(x)
        prior_logprob = self.prior.log_prob(zs[-1]).view(x.size(0), -1).sum(1)
        return zs, prior_logprob, log_det

    def backward(self, z):
        xs, log_det = self.flow.backward(z)
        return xs, log_det
    
    def sample(self, num_samples):
        z = self.prior.sample((num_samples,))
        xs, _ = self.flow.backward(z)#should i update this to be the approximate posterior??????????????????
        return xs
    
class AffineConstantFlow(nn.Module):
    """ 
    Scales + Shifts the flow by (learned) constants per dimension.
    In NICE paper there is a Scaling layer which is a special case of this where t is None
    """
    def __init__(self, dim, scale=True, shift=True):
        super().__init__()
        self.s = nn.Parameter(torch.randn(1, dim, requires_grad=True)) if scale else None
        self.t = nn.Parameter(torch.randn(1, dim, requires_grad=True)) if shift else None
        
    def forward(self, x):
        s = self.s if self.s is not None else x.new_zeros(x.size())
        t = self.t if self.t is not None else x.new_zeros(x.size())
        z = x * torch.exp(s) + t
        log_det = torch.sum(s, dim=1)
        return z, log_det
    
    def backward(self, z):
        s = self.s if self.s is not None else z.new_zeros(z.size())
        t = self.t if self.t is not None else z.new_zeros(z.size())
        x = (z - t) * torch.exp(-s)
        log_det = torch.sum(-s, dim=1)
        return x, log_det
    
class stacked_NVP(nn.Module):
    def __init__(self, d, k, hidden, n):
        super().__init__()
        self.bijectors = nn.ModuleList([
            R_NVP(d, k, hidden=hidden) for _ in range(n)
        ])
        self.flips = [True if i%2 else False for i in range(n)]
        
    def forward(self, x):
        log_jacobs = []
        
        for bijector, f in zip(self.bijectors, self.flips):
            x, log_pz, lj = bijector(x, flip=f)
            log_jacobs.append(lj)
        
        return x, log_pz, sum(log_jacobs)
    
    def inverse(self, z):
        for bijector, f in zip(reversed(self.bijectors), reversed(self.flips)):
            z = bijector.inverse(z, flip=f)
        return z
    
class R_NVP(nn.Module):
    def __init__(self, d, k, hidden):
        super().__init__()
        self.d, self.k = d, k
        self.sig_net = nn.Sequential(
                    nn.Linear(k, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, d - k))

        self.mu_net = nn.Sequential(
                    nn.Linear(k, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, d - k))

    def forward(self, x, flip=False):
        x1, x2 = x[:, :self.k], x[:, self.k:] 

        if flip:
            x2, x1 = x1, x2
        
        # forward
        sig = self.sig_net(x1)
        z1, z2 = x1, x2 * torch.exp(sig) + self.mu_net(x1)
        
        if flip:
            z2, z1 = z1, z2
        
        z_hat = torch.cat([z1, z2], dim=-1)

        log_pz = base_dist.log_prob(z_hat)
        log_jacob = sig.sum(-1)
        
        return z_hat, log_pz, log_jacob
    
    def inverse(self, Z, flip=False):
        z1, z2 = Z[:, :self.k], Z[:, self.k:] 
        
        if flip:
            z2, z1 = z1, z2
        
        x1 = z1
        x2 = (z2 - self.mu_net(z1)) * torch.exp(-self.sig_net(z1))
        
        if flip:
            x2, x1 = x1, x2
        return torch.cat([x1, x2], -1)

In [88]:
step_size= 3
batch = 128
index_step_length = 31
epochs = 50
#---------------------------------------------------------------------------------------------------------------------------------
labels, X, Y, XX, YY = Splitting_dataset(data, step_size)
demo = VAE(index_step_length,h_dim=7,z_dim=31)
demo.double()
optimizer = torch.optim.RMSprop(demo.parameters(), lr=1e-4)

In [89]:
idx = 0

anomaly_history = []
loss_history = []
avgSum = 0
avgCount = 0

for epoch in range(epochs):
    for i in range(len(XX)):
        localX = torch.tensor(XX[i])
        
        
        recon, mu, logvar = demo(localX)
        loss = loss_fn(recon, localX, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        idx = idx + 1

        
        avgSum = avgSum + torch.mean(loss/batch)
        avgCount = avgCount + 1
        anomaly_score = torch.mean(localX/recon)

        if idx%10 == 0:
            loss_history.append(avgSum/avgCount)
            anomaly_history.append(anomaly_score)
            avgSum = 0
            avgCount = 0

        if idx%100 == 0:
            print("Epoch[{}/{}] Loss: {:.3f}".format(epoch+1, epochs, loss.data.item()/batch))

plt.plot(loss_history,'g-',label='h 10,z 2')

tensor([[-0.9327,  0.1387,  0.7461, -0.0986,  1.4332,  0.9343, -0.2574,  1.3938,
         -0.2541,  1.7783, -0.8193,  1.0418, -1.3503,  0.3057, -2.0342,  0.7451,
          0.3038, -1.5743, -0.8273,  1.0899, -0.4213,  0.5909,  1.3237, -1.2872,
          0.8823, -0.5909, -0.9916, -0.2545,  0.6067, -0.2910,  2.1028],
        [ 0.0496, -0.0623,  1.2893, -0.6281, -0.9708, -2.1139,  0.1517, -0.7570,
          0.3810, -0.1513,  0.1013,  0.3297, -1.4870, -1.9221,  0.4075, -2.3025,
         -1.8509, -1.0612, -1.3508,  0.5216,  1.4911,  0.5932,  0.2857, -0.0171,
          1.1302,  0.1357, -1.2503,  0.2010, -0.2165, -0.7769,  0.5447],
        [ 0.2894, -0.7946, -2.3291, -1.8148,  0.7920, -1.8095, -0.6785, -1.0825,
         -0.3049, -1.0473, -0.5414, -1.5631, -1.7942, -0.7918, -1.2143,  0.3409,
         -1.1434,  1.1907, -0.5282,  0.0371, -0.4544, -0.0236, -0.7175,  0.7032,
         -0.7953, -0.0548, -0.5624,  0.7704, -0.1357,  0.7720, -1.6490]],
       dtype=torch.float64, grad_fn=<AddBackward0>)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not list